In [ ]:
#import numpy as np
#import scipy as sp
#import matplotlib as mpl
#import matplotlib.cm as cm
#import matplotlib.pyplot as plt
#import pandas as pd
#import time
#pd.set_option('display.width', 500)
#pd.set_option('display.max_columns', 100)
#pd.set_option('display.notebook_repr_html', True)

In [ ]:
'''
Based on SymSpell:

Originally written in C#:

// SymSpell: 1 million times faster through Symmetric Delete spelling correction algorithm
//
// The Symmetric Delete spelling correction algorithm reduces the complexity of edit candidate generation and dictionary lookup 
// for a given Damerau-Levenshtein distance. It is six orders of magnitude faster and language independent.
// Opposite to other algorithms only deletes are required, no transposes + replaces + inserts.
// Transposes + replaces + inserts of the input term are transformed into deletes of the dictionary term.
// Replaces and inserts are expensive and language dependent: e.g. Chinese has 70,000 Unicode Han characters!
//
// Copyright (C) 2015 Wolf Garbe
// Version: 3.0
// Author: Wolf Garbe <wolf.garbe@faroo.com>
// Maintainer: Wolf Garbe <wolf.garbe@faroo.com>
// URL: http://blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/
// Description: http://blog.faroo.com/2012/06/07/improved-edit-distance-based-spelling-correction/
//
// License:
// This program is free software; you can redistribute it and/or modify
// it under the terms of the GNU Lesser General Public License, 
// version 3.0 (LGPL-3.0) as published by the Free Software Foundation.
// http://www.opensource.org/licenses/LGPL-3.0
//
// Usage: single word + Enter:  Display spelling suggestions
//        Enter without input:  Terminate the program
'''
pass

In [1]:
#import re, collections
import re

In [2]:
max_edit_distance = 3
dictionary = {}
longest_word_length = 0

In [3]:
def get_deletes_list(w):
    '''given a word, derive strings with up to max_edit_distance characters deleted'''
    deletes = []
    queue = [w]
    for d in range(max_edit_distance):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes

In [ ]:
# get_deletes_list("tomorrow")

In [4]:
def create_dictionary_entry(w):
    '''add word and its derived deletions to dictionary'''
    # check if word is already in dictionary
    # dictionary entries are in the form: (list of suggested corrections, frequency of word in corpus)
    global longest_word_length
    new_real_word_added = False
    if w in dictionary:
        dictionary[w] = (dictionary[w][0], dictionary[w][1] + 1)  # increment count of word in corpus
    else:
        dictionary[w] = ([], 1)  
        longest_word_length = max(longest_word_length, len(w))
        
    if dictionary[w][1]==1:
        # first appearance of word in corpus
        # n.b. word may already be in dictionary as a derived word (deleting character from a real word)
        # but counter of frequency of word in corpus is not incremented in those cases)
        new_real_word_added = True
        deletes = get_deletes_list(w)
        for item in deletes:
            if item in dictionary:
                # add (correct) word to delete's suggested correction list if not already there
                if item not in dictionary[item][0]:
                    dictionary[item][0].append(w)
            else:
                dictionary[item] = ([w], 0)  # note frequency of word in corpus is not incremented
        
    return new_real_word_added

In [5]:
def create_dictionary(fname):
    word_count = 0
    print "Creating dictionary..." 
    
    with open(fname) as file:    
        for line in file:
            words = re.findall('[a-z]+', line.lower())  # separate by words by non-alphabetical characters      
            for word in words:
                if create_dictionary_entry(word):
                    word_count += 1
                    
    print "total unique words in corpus: %i" % word_count
    print "total items in dictionary (corpus words and deletions): %i" % len(dictionary)
    print "  edit distance for deletions: %i" % max_edit_distance
    print "  length of longest word in corpus: %i" % longest_word_length
        
    return dictionary

In [6]:
%%time
create_dictionary("/Users/K-Lo/Desktop/big.txt")
pass

Creating dictionary...
total unique words in corpus: 29157
total items in dictionary (corpus words and deletions): 2151998
  edit distance for deletions: 3
  length of longest word in corpus: 18
CPU times: user 26.1 s, sys: 513 ms, total: 26.6 s
Wall time: 26.6 s


<div class="alert alert-info">
<b>For testing:</b> <p>
Can look up a specific entry in the dictionary below. <br>
shows (possible corrections, and frequency that entry itself is in corpus - 0 if not a real word) <br>
Note: will return key error if there are no corrections (because we are accessing dictionary directly here)
</div>

In [7]:
dictionary["essential"]

(['essentially', 'essentials'], 92)

In [8]:
dictionary["wack"]

(['wrack'], 0)

In [9]:
def dameraulevenshtein(seq1, seq2):
    """Calculate the Damerau-Levenshtein distance between sequences.

    Source: http://mwh.geek.nz/2009/04/26/python-damerau-levenshtein-distance/
    
    This distance is the number of additions, deletions, substitutions,
    and transpositions needed to transform the first sequence into the
    second. Although generally used with strings, any sequences of
    comparable objects will work.

    Transpositions are exchanges of *consecutive* characters; all other
    operations are self-explanatory.

    This implementation is O(N*M) time and O(M) space, for N and M the
    lengths of the two sequences.

    >>> dameraulevenshtein('ba', 'abc')
    2
    >>> dameraulevenshtein('fee', 'deed')
    2

    It works with arbitrary sequences too:
    >>> dameraulevenshtein('abcd', ['b', 'a', 'c', 'd', 'e'])
    2
    """
    # codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    # Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1 matrix.
    # However, only the current and two previous rows are needed at once,
    # so we only store those.
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    for x in xrange(len(seq1)):
        # Python lists wrap around for negative indices, so put the
        # leftmost column at the *end* of the list. This matches with
        # the zero-indexed strings and saves extra calculation.
        twoago, oneago, thisrow = oneago, thisrow, [0] * len(seq2) + [x + 1]
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)
            # This block deals with transpositions
            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
    return thisrow[len(seq2) - 1]

In [10]:
def get_suggestions(string, silent=False):
    '''return list of suggested corrections for potentially incorrectly spelled word'''
    if (len(string) - longest_word_length) > max_edit_distance:
        if not silent:
            print "no items in dictionary within maximum edit distance"
        return []
    
    # suggestions = []
    # s_dictionary = {}
    suggest_dict = {}
    
    queue = [string]
    q_dictionary = {}  # items other than string that we've checked
    
    while len(queue)>0:
        q_item = queue[0]  # pop
        # print "processing '%s'" % q_item
        queue = queue[1:]
        
        # process queue item
        if (q_item in dictionary) and (q_item not in suggest_dict):
            if (dictionary[q_item][1]>0):
            # word is in dictionary, and is a word from the corpus, and not already in suggestion list
            # so add to suggestion dictionary, indexed by the word with value (frequency in corpus, edit distance)
            # note q_items that are not the input string are shorter than input string 
            # since only deletes are added (unless manual dictionary corrections are added)
                assert len(string)>=len(q_item)
                suggest_dict[q_item] = (dictionary[q_item][1], len(string) - len(q_item))
            
            ## the suggested corrections for q_item as stored in dictionary (whether or not
            ## q_item itself is a valid word or merely a delete) can be valid corrections
            for sc_item in dictionary[q_item][0]:
                if (sc_item not in suggest_dict):
                    # compute edit distance
                    # if len(sc_item)==len(q_item):
                    #    item_dist = len(string) - len(q_item)
                    # suggested items should always be longer (unless manual corrections are added)
                    assert len(sc_item)>len(q_item)
                    # q_items that are not input should be shorter than original string 
                    # (unless manual corrections added)
                    assert len(q_item)<=len(string)
                    if len(q_item)==len(string):
                        assert q_item==string
                        item_dist = len(sc_item) - len(q_item)
                    #elif len(q_item)==len(string):
                        # a suggestion could be longer or shorter than original string (bug in original FAROO?)
                        # if suggestion is from string's suggestion list, sc_item will be longer
                        # if suggestion is from a delete's suggestion list, sc_item may be shorter
                    #   item_dist = abs(len(sc_item) - len(q_item))
                    #else:
                    # check in original code, but probably not necessary because string has already checked
                    assert sc_item!=string
                    
                    # calculate edit distance using, for example, Damerau-Levenshtein distance
                    item_dist = dameraulevenshtein(sc_item, string)
                    
                    if item_dist<=max_edit_distance:
                        assert sc_item in dictionary  # should already be in dictionary if in suggestion list
                        suggest_dict[sc_item] = (dictionary[sc_item][1], item_dist)
        
        # now generate deletes (e.g. a substring of string or of a delete) from the queue item
        # as additional items to check -- add to end of queue
        assert len(string)>=len(q_item)
        if (len(string)-len(q_item))<max_edit_distance and len(q_item)>1:
            for c in range(len(q_item)): # character index        
                word_minus_c = q_item[:c] + q_item[c+1:]
                if word_minus_c not in q_dictionary:
                    queue.append(word_minus_c)
                    q_dictionary[word_minus_c] = None  # arbitrary value, just to identify we checked this
             
    # queue is now empty: convert suggestions in dictionary to list for output
    
    if not silent:
        print "number of possible corrections: %i" %len(suggest_dict)
        print "  edit distance for deletions: %i" % max_edit_distance
    
    # output option 1
    # sort results by ascending order of edit distance and descending order of frequency
    #     and return list of suggested corrections only:
    # return sorted(suggest_dict, key = lambda x: (suggest_dict[x][1], -suggest_dict[x][0]))

    # output option 2
    # return list of suggestions with (correction, (frequency in corpus, edit distance)):
    as_list = suggest_dict.items()
    return sorted(as_list, key = lambda (term, (freq, dist)): (dist, -freq))

    '''
    Option 1:
    get_suggestions("file")
    ['file', 'five', 'fire', 'fine', ...]
    
    Option 2:
    get_suggestions("file")
    [('file', (5, 0)),
     ('five', (67, 1)),
     ('fire', (54, 1)),
     ('fine', (17, 1))...]  
    '''

<div class="alert alert-info">
<b>For testing:</b><p>
Type in word to correct below, to test and get whole list of possible suggestions.
</div>

In [81]:
%%time
get_suggestions("mittens")

number of possible corrections: 103
  edit distance for deletions: 3
CPU times: user 20 ms, sys: 7.7 ms, total: 27.7 ms
Wall time: 22.1 ms


[('matters', (136, 2)),
 ('bitten', (13, 2)),
 ('kitten', (7, 2)),
 ('listens', (2, 2)),
 ('battens', (1, 2)),
 ('smitten', (1, 2)),
 ('matter', (365, 3)),
 ('sitting', (269, 3)),
 ('minutes', (146, 3)),
 ('written', (117, 3)),
 ('miles', (110, 3)),
 ('citizens', (109, 3)),
 ('letters', (108, 3)),
 ('listen', (100, 3)),
 ('cities', (77, 3)),
 ('bitter', (47, 3)),
 ('masters', (37, 3)),
 ('intense', (34, 3)),
 ('witness', (33, 3)),
 ('attend', (29, 3)),
 ('mistress', (24, 3)),
 ('fitted', (23, 3)),
 ('mines', (22, 3)),
 ('fitting', (21, 3)),
 ('miners', (19, 3)),
 ('mitenka', (16, 3)),
 ('tens', (16, 3)),
 ('sisters', (16, 3)),
 ('intent', (13, 3)),
 ('mothers', (12, 3)),
 ('mutton', (11, 3)),
 ('bites', (10, 3)),
 ('sites', (9, 3)),
 ('omitted', (9, 3)),
 ('buttons', (8, 3)),
 ('fitness', (8, 3)),
 ('rotten', (8, 3)),
 ('attends', (8, 3)),
 ('matrena', (8, 3)),
 ('intend', (8, 3)),
 ('titles', (7, 3)),
 ('pitted', (6, 3)),
 ('matted', (6, 3)),
 ('omitting', (5, 3)),
 ('intends', (5, 3)

In [ ]:
%%time
#benchmark timing
for i in range(1000):
    get_suggestions("acamodation", silent=True)

In [ ]:
%%time
#benchmark timing
for i in range(1000):
    get_suggestions("acomodation", silent=True)

In [ ]:
%%time
#benchmark timing
for i in range(1000):
    get_suggestions("hous", silent=True)

In [12]:
# get best word
def best_word(s, silent=False):
    try:
        return get_suggestions(s, silent)[0]
    except:
        return None

<div class="alert alert-info">
<b>For testing:</b><p>
Type in word to correct below, to test and get most suggested word.
</div>

In [13]:
best_word("hello")

number of possible corrections: 337
  edit distance for deletions: 3


('hello', (1, 0))

In [17]:
def correct_document(fname):
    with open(fname) as file:
        doc_word_count = 0
        corrected_word_count = 0
        unknown_word_count = 0
        print "Finding misspelled words in your document..." 
        
        for i, line in enumerate(file):
            doc_words = re.findall('[a-z]+', line.lower())  # separate by words by non-alphabetical characters      
            for doc_word in doc_words:
                doc_word_count += 1
                suggestion = best_word(doc_word, silent=True)
                if suggestion is None:
                    print "In line %i, the word < %s > was not found (no suggested correction)" % (i, doc_word)
                    unknown_word_count += 1
                elif suggestion[0]!=doc_word:
                    print "In line %i, %s: suggested correction is < %s >" % (i, doc_word, suggestion[0])
                    corrected_word_count += 1
        
    print "-----"
    print "total words checked: %i" % doc_word_count
    print "total unknown words: %i" % unknown_word_count
    print "total potential errors found: %i" % corrected_word_count

    return

<div class="alert alert-info">
<b>For testing:</b><p>
Provide text file to correct, and give all best word suggestions (word level only) for errors.
</div>

In [18]:
correct_document("/Users/K-Lo/Desktop/test.txt")

Finding misspelled words in your document...
-----
total words checked: 16
total unknown words: 0
total potential errors found: 0


In [19]:
# from http://www.columbia.edu/acis/cria/rosenberg/sample/
correct_document("/Users/K-Lo/Desktop/OCRsample.txt")

Finding misspelled words in your document...
In line 3, taiths: suggested correction is < faith >
In line 11, the word < oonipiittee > was not found (no suggested correction)
In line 13, tj: suggested correction is < to >
In line 13, mnnff: suggested correction is < snuff >
In line 13, gjpt: suggested correction is < get >
In line 15, bh: suggested correction is < by >
In line 15, snc: suggested correction is < sac >
In line 15, uth: suggested correction is < th >
In line 15, unuer: suggested correction is < under >
In line 20, mthiitt: suggested correction is < thirty >
In line 21, cas: suggested correction is < was >
In line 22, pythian: suggested correction is < scythian >
In line 26, brainin: suggested correction is < brain >
In line 27, jfl: suggested correction is < of >
In line 28, ji: suggested correction is < i >
In line 28, stice: suggested correction is < stick >
In line 28, blaci: suggested correction is < black >
In line 28, eug: suggested correction is < dug >
In line 28,

<div class="alert alert-warning">
Serial code for context-based improvement <p>
Using same corpus to generate, but could consider pre-manufactured lists<br>
Could integrate with code above when generating main dictionary
</div>

In [20]:
first_of_two = {}
first_counts = {}
last_of_two = {}
last_2counts = {}
last_of_three = {}
last_3counts = {}

In [21]:
def create_trigram_dict(fname):
    '''creates bigram dictionaries, and 1 trigram dictionary for (__, __, word)'''
    word_count = 0
    print "Creating trigram database..." 
    
    word1 = ""
    word2 = ""
    
    with open(fname) as file:    
        for line in file:
            words = re.findall('[a-z]+', line.lower())  # separate by words by non-alphabetical characters      
            for word in words:
                word_count += 1
                if len(word1)==0:
                    word1 = word
                elif len(word2)==0:
                    word2 = word
                    first_of_two[word1] = [word2]
                    first_counts[word1 + " " + word2] = 1
                    last_of_two[word2] = [word1]
                    last_2counts[word1 + " " + word2] = 1
                else:
                    #third word onwards
                    if word2 in first_of_two:
                        if word in first_of_two[word2]:
                            first_counts[word2 + " " + word] += 1
                        else:
                            first_of_two[word2].append(word)
                            first_counts[word2 + " " + word] = 1
                    else:
                        first_of_two[word2] = [word]
                        first_counts[word2 + " " + word] = 1
                        
                    if word in last_of_two:
                        if word2 in last_of_two[word]:
                            last_2counts[word2 + " " + word] += 1
                        else:
                            last_of_two[word].append(word2)
                            last_2counts[word2 + " " + word] = 1
                    else:
                        last_of_two[word] = [word2]
                        last_2counts[word2 + " " + word] = 1
                        
                    back_two = word1 + " " + word2
                    if word in last_of_three:
                        if back_two in last_of_three[word]:
                            last_3counts[back_two + " " + word] += 1
                        else:
                            last_of_three[word].append(back_two)
                            last_3counts[back_two + " " + word] = 1
                    else:
                        last_of_three[word] = [back_two]
                        last_3counts[back_two + " " + word] = 1
                    
                    #increment words
                    word1 = word2
                    word2 = word
    
    print "total unique words in corpus: %i" % word_count
    print "total number of unique first words of a pair: %i" % len(first_counts)
    print "total number of unique second words of a pair: %i" % len(last_2counts)
    print "total number of unique last words of a trigram: %i" % len(last_3counts)
        
    return True

In [22]:
%%time
create_trigram_dict("/Users/K-Lo/Desktop/big.txt")

Creating trigram database...
total unique words in corpus: 1105285
total number of unique first words of a pair: 378951
total number of unique second words of a pair: 378951
total number of unique last words of a trigram: 818180
CPU times: user 4min 12s, sys: 2.81 s, total: 4min 15s
Wall time: 4min 15s


True

In [ ]:
# first_of_two

In [ ]:
# first_counts

In [ ]:
# last_of_two

In [ ]:
# last_2counts

In [ ]:
# last_of_three

In [ ]:
# last_3counts

In [ ]:
# last_of_three["there"]

In [ ]:
# "here and" in last_of_three["there"]

In [ ]:
# "here and" in last_of_three["their"]

In [23]:
last_3counts["here and there"]

28

In [ ]:
# "and" in last_of_two["there"]

In [ ]:
# "and" in last_of_two["their"]

In [24]:
last_2counts["and there"]

227

In [25]:
last_2counts["and their"]

147

In [26]:
def best_context(w1, w2, w_list, k):
    '''determine most frequent trigram (greater than threshold k) for every w in w_list: (w1, w2, w)'''
    # w_list comprises a list of tuples (w, (other info))
    leading = w1 + " " + w2
    
    best_word = ""
    trigram_found = False
    best_tri_count = 0
    best_bi_count = 0
    
    for item in w_list:
        w = item[0]
        if (leading in last_of_three[w]) and (last_3counts[leading + " " + w]>max(k, best_tri_count)):
            best_tri_count = last_3counts[leading + " " + w]
            best_word = w
            trigram_found = True
        elif (not trigram_found) and (w2 in last_of_two[w]) and (last_2counts[w2 + " " + w]>max(k, best_bi_count)):
            # check bigrams only if no trigram candidates found
            best_bi_count = last_2counts[w2 + " " + w]
            best_word = w
    
    return best_word

In [77]:
def correct_document_context(fname, context_threshold=5, num_word_suggestions=5000):
    with open(fname) as file:
        doc_word_count2 = 0
        corrected_word_count = 0
        unknown_word_count = 0
        mismatches = 0
        print "Finding misspelled words in your document..." 
        
        word1 = ""
        word2 = ""
        first_two = True
        
        for i, line in enumerate(file):
            doc_words2 = re.findall('[a-z]+', line.lower())  # separate by words by non-alphabetical characters      
            for doc_word in doc_words2:
                suggestion_w = None
                suggestion_c = None
                doc_word_count2 += 1
                
                if len(word1)==0:
                    word1 = doc_word
                elif len(word2)==0:
                    word2 = doc_word
                else:
                    first_two = False
                
                # character level correction
                list_suggestions_byword = get_suggestions(doc_word, silent=True)
                if len(list_suggestions_byword)>num_word_suggestions:
                    list_suggestions_byword = list_suggestions_byword[0:num_word_suggestions]
                if len(list_suggestions_byword)>0:
                    suggestion_w = list_suggestions_byword[0][0]  # string of most frequent word tuple
                    
                    # context correction on all potential suggestions
                    # note if there are no suggestions, we do not perform context correction

                    # current algorithm - choose the best trigram if count is over threshold
                    # if none then choose best bigram (doc_word as last word) if count is over threshold
                    # if none then use best character suggestion
                    if not first_two:
                        suggestion_c = best_context(word1, word2, list_suggestions_byword, context_threshold)
                        # print suggestion_c
                        if len(suggestion_c)==0:
                            suggestion_c = suggestion_w
                    else:
                        suggestion_c = suggestion_w
                
                if suggestion_w is None:
                    print "In line %i, the word < %s > was not found (no suggested corrections)" % (i, doc_word)
                    unknown_word_count += 1
                elif (suggestion_w!=doc_word) or (suggestion_c!=doc_word):
                    print "In line %i, (%s %s) %s: word level correction is < %s >, context correction is < %s >" \
                              % (i, word1, word2, doc_word, suggestion_w, suggestion_c)
                    corrected_word_count += 1
                    if suggestion_w!=suggestion_c:
                        mismatches += 1
                                        
                word1 = word2
                word2 = doc_word
        
    print "-----"
    print "total words checked: %i" % doc_word_count2
    print "total unknown words: %i" % unknown_word_count
    print "total potential errors found: %i" % corrected_word_count
    print "total mismatches (word level vs. context): %i" % mismatches
        
    return

<div class="alert alert-info">
<b>For testing:</b><p>
Provide text file to correct, and give all best word suggestions (word level & context level) for all known words.
</div>

In [72]:
#test.txt

#this is a test
#this is a test
#here is a test
#this is ax test
#this is za test

In [73]:
correct_document_context("/Users/K-Lo/Desktop/test.txt")

Finding misspelled words in your document...
In line 0, (is a) test: word level correction is < test >, context correction is < very >
In line 1, (is a) test: word level correction is < test >, context correction is < very >
In line 2, (is a) test: word level correction is < test >, context correction is < very >
In line 3, (this is) ax: word level correction is < ax >, context correction is < a >
In line 4, (this is) za: word level correction is < a >, context correction is < a >
-----
total words checked: 20
total unknown words: 0
total potential errors found: 5
total mismatches (word level vs. context): 4


In [47]:
last_3counts["is a very"]

26

In [48]:
last_3counts["is a test"]

KeyError: 'is a test'

In [49]:
last_3counts["a test this"]

KeyError: 'a test this'

In [50]:
last_3counts["a test tube"]

3

In [74]:
correct_document_context("/Users/K-Lo/Desktop/OCRsample.txt")

Finding misspelled words in your document...
In line 3, (ny di) taiths: word level correction is < faith >, context correction is < faith >
In line 11, the word < oonipiittee > was not found (no suggested corrections)
In line 11, (oonipiittee to) pay: word level correction is < pay >, context correction is < a >
In line 11, (pay the) expenses: word level correction is < expenses >, context correction is < expense >
In line 12, (expenses and) charges: word level correction is < charges >, context correction is < horses >
In line 12, (preparing and) filing: word level correction is < filing >, context correction is < having >
In line 12, (filing the) printed: word level correction is < printed >, context correction is < united >
In line 13, (record and) brief: word level correction is < brief >, context correction is < are >
In line 13, (julius and) t: word level correction is < t >, context correction is < the >
In line 13, (and t) tj: word level correction is < to >, context correction

In [75]:
correct_document_context("/Users/K-Lo/Desktop/usingengsample.txt")

Finding misspelled words in your document...
In line 0, (started my) schooling: word level correction is < choosing >, context correction is < choosing >
In line 0, (the majority) did: word level correction is < did >, context correction is < in >
In line 0, (majority did) in: word level correction is < in >, context correction is < not >
In line 0, (in my) area: word level correction is < area >, context correction is < dear >
In line 0, (my area) at: word level correction is < at >, context correction is < and >
In line 0, (at the) local: word level correction is < local >, context correction is < road >
In line 0, (the local) primarry: word level correction is < primary >, context correction is < primary >
In line 0, (school i) then: word level correction is < then >, context correction is < don >
In line 0, (i then) went: word level correction is < went >, context correction is < he >
In line 0, (to the) local: word level correction is < local >, context correction is < law >
In li

In [80]:
correct_document_context("/Users/K-Lo/Desktop/usingengsample.txt", 50, 5)

Finding misspelled words in your document...
In line 0, (started my) schooling: word level correction is < choosing >, context correction is < choosing >
In line 0, (the local) primarry: word level correction is < primary >, context correction is < primary >
In line 0, (the local) secondarry: word level correction is < secondary >, context correction is < secondary >
In line 0, (school and) recieved: word level correction is < received >, context correction is < received >
In line 0, (in english) maths: word level correction is < baths >, context correction is < baths >
In line 0, (english maths) phisics: word level correction is < physics >, context correction is < physics >
In line 0, (geography art) graphical: word level correction is < graphics >, context correction is < graphics >
In line 0, (art graphical) comunication: word level correction is < communication >, context correction is < communication >
In line 0, (philosophy of) religeon: word level correction is < religion >, co